In [1]:
#requires tensorflow2
import tensorflow as tf
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import keras
import numpy as np
from ads_utils import load_data, plot, Environment

In [2]:
INITIAL_BALANCE = 10_000
PAST_TICKS = 5

In [3]:
answers = [INITIAL_BALANCE]
actions = []
portfolio_values = [INITIAL_BALANCE]
balance = INITIAL_BALANCE
prices = []

In [ ]:
for i in range(6):
    data = load_data([i for i in range(6-i,6-i+4)])
    dataset = np.reshape(data['close'].values,(-1,1))
    scaled_dataset = dataset
    train= scaled_dataset[:int(scaled_dataset.shape[0]*0.75)]
    valid = scaled_dataset[int(scaled_dataset.shape[0]*0.75)-60:]
    from sklearn.preprocessing import MinMaxScaler
    sc = MinMaxScaler(feature_range = (0, 1))
    train2 = sc.fit_transform(train)
    valid2 = sc.transform(valid)
    x_train,y_train,x_test,y_test = [],[],[],[]
    for j in range(60,train2.shape[0]):
        x_train.append(train2[j-60:j,0])
        y_train.append(train2[j,0])
    for z in range(60,valid2.shape[0]):
        x_test.append(valid2[z-60:z,0])
        y_test.append(valid2[z,0])
    x_train, y_train,x_test,y_test = np.array(x_train), np.array(y_train),np.array(x_test),np.array(y_test)
    x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
    x_test=np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
    from keras.models import Sequential
    from keras.layers import Dense, Dropout,LSTM
    from keras.optimizers import Adam
    model = Sequential()
    model.add(LSTM(units=100,input_shape=(x_train.shape[1],1),return_sequences=True))
    model.add(LSTM(units=100))
    model.add(Dropout(0.4))
    model.add(Dense(1))
    ADAM = Adam(0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    model.compile(loss='mean_squared_error', optimizer=ADAM)
    history = model.fit(x_train,y_train,epochs=20,batch_size=72,validation_data=(x_test,y_test),verbose=1,shuffle=False)
    predicted_stock_price = sc.inverse_transform(model.predict(x_test))
    actual_stock_price = sc.inverse_transform(y_test.reshape((-1,1)))
    for j in range(len(actual_stock_price)):
        prices.append(actual_stock_price[j][0])
    for j in range(1,len(predicted_stock_price)):
        if predicted_stock_price[j]>actual_stock_price[j-1]:
            #go long
            balance += actual_stock_price[j]-actual_stock_price[j-1]
            actions.append(2)
        else:
            #go short
            balance += actual_stock_price[j-1]-actual_stock_price[j]
            actions.append(0)
        portfolio_values.append(float(balance))
    plot(prices=prices,target_positions = actions,portfolio_values = portfolio_values)

Epoch 1/20
447/447 [==============================] - 38s 86ms/step - loss: 0.1217 - val_loss: 0.0127
Epoch 2/20
447/447 [==============================] - 38s 84ms/step - loss: 0.0077 - val_loss: 0.0059
Epoch 3/20
447/447 [==============================] - 38s 84ms/step - loss: 0.0068 - val_loss: 0.0033
Epoch 4/20
447/447 [==============================] - 39s 87ms/step - loss: 0.0058 - val_loss: 0.0017
Epoch 5/20
447/447 [==============================] - 39s 87ms/step - loss: 0.0051 - val_loss: 0.0011
Epoch 6/20
447/447 [==============================] - 39s 86ms/step - loss: 0.0045 - val_loss: 2.8265e-04
Epoch 7/20
447/447 [==============================] - 39s 88ms/step - loss: 0.0040 - val_loss: 3.0843e-04
Epoch 8/20
447/447 [==============================] - 39s 86ms/step - loss: 0.0038 - val_loss: 2.5980e-04
Epoch 9/20
447/447 [==============================] - 39s 87ms/step - loss: 0.0035 - val_loss: 5.9796e-04
Epoch 10/20
447/447 [==============================] - 40s 89ms/st